## Importing Spacy

In [3]:
# !conda install -c conda-forge spacy
# !python -m spacy download en_core_web_sm
## I recommend the one above, because the following is more accurate but less efficient
# !python -m spacy download en_core_web_lg

In [4]:
import spacy


nlp = spacy.load("en_core_web_sm")
# You can also load en_core_web_lg that has an higher accuracy but it's less efficient
# nlp = spacy.load("en_core_web_lg")



In [5]:
print(nlp.pipeline)

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7f86ff1ef9f0>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7f86ff19f950>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7f86ff1b2100>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7f86ff3b3be0>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7f86ff1b4cc0>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7f86ff161440>)]


In [6]:
# Process sentences 'Hello, world. Antonio is learning Python.' using spaCy
doc = nlp(u"Hello, world. Antonio is learning Python.")

In [7]:
for token in doc:
    print(token.text)

Hello
,
world
.
Antonio
is
learning
Python
.


In [8]:
# Get first token of the processed document
token = doc[0]
print(token)

# Print sentences (one sentence per line)
for sent in doc.sents:
    print(sent)

Hello
Hello, world.
Antonio is learning Python.


In [9]:
tokens = nlp("Let's go to N.Y.!")

In [10]:
for token in tokens:
    print(token.text)

Let
's
go
to
N.Y.
!


As you have seen, using `nlp`, that comes from `spacy.load("en_core_web_sm")`, you get the tokenized version of the sentence. If you want only the instance of the `Tokenizer` class, you can run:

In [11]:
tokenizer = nlp.tokenizer
type(tokenizer)

spacy.tokenizer.Tokenizer

If you want to instantiate a custom one, with rules and prefixes and so on:

In [12]:
from spacy.tokenizer import Tokenizer

tokenizer = Tokenizer(vocab=nlp.vocab)

The tokenizer defined above contains only english rules.
Let's test it on "Let's go to N.Y.!"

In [13]:
tokens = tokenizer("Let's go to N.Y.!")
for token in tokens:
    print(token)



Let's
go
to
N.Y.!


As you can see here, it doesn't handle the exceptions about the dots. So we can add rules for this!

In [14]:
prefix_re = spacy.util.compile_prefix_regex(nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(nlp.Defaults.prefixes)

In [15]:
tokenizer = Tokenizer(
    vocab=nlp.vocab, prefix_search=prefix_re.search, suffix_search=suffix_re.search
)

In [16]:
tokens = tokenizer("Let's go to N.Y.!")
for token in tokens:
    print(token)

Let's
go
to
N.Y.
!


You can also check the exceptions the tokenizer can handle:

In [17]:
from spacy.lang.en.tokenizer_exceptions import TOKENIZER_EXCEPTIONS

TOKENIZER_EXCEPTIONS.values()

dict_values([[{65: ' '}], [{65: '\t'}], [{65: '\\t'}], [{65: '\n'}], [{65: '\\n'}], [{65: '—'}], [{65: '\xa0', 67: '  '}], [{65: "'"}], [{65: '\\")'}], [{65: '<space>'}], [{65: "''"}], [{65: 'C++'}], [{65: 'a.'}], [{65: 'b.'}], [{65: 'c.'}], [{65: 'd.'}], [{65: 'e.'}], [{65: 'f.'}], [{65: 'g.'}], [{65: 'h.'}], [{65: 'i.'}], [{65: 'j.'}], [{65: 'k.'}], [{65: 'l.'}], [{65: 'm.'}], [{65: 'n.'}], [{65: 'o.'}], [{65: 'p.'}], [{65: 'q.'}], [{65: 'r.'}], [{65: 's.'}], [{65: 't.'}], [{65: 'u.'}], [{65: 'v.'}], [{65: 'w.'}], [{65: 'x.'}], [{65: 'y.'}], [{65: 'z.'}], [{65: 'ä.'}], [{65: 'ö.'}], [{65: 'ü.'}], [{65: '[:'}], [{65: ":'("}], [{65: '=3'}], [{65: '>.>'}], [{65: ':))'}], [{65: '(='}], [{65: ':X'}], [{65: 'O.O'}], [{65: '<333'}], [{65: '<33'}], [{65: ':-o'}], [{65: '=]'}], [{65: ':-('}], [{65: '(-;'}], [{65: '><(((*>'}], [{65: 'o.O'}], [{65: '(-:'}], [{65: 'O_O'}], [{65: 'o.0'}], [{65: '</3'}], [{65: '=['}], [{65: '=('}], [{65: ':-/'}], [{65: '0_0'}], [{65: 'V_V'}], [{65: ':-O'}], [{65: 

In [18]:
tokens = tokenizer("This is a $STOCK.")
for token in tokens:
    print(token)

This
is
a
$
STOCK.


You can add special prefixes in the form of regex by doing:

In [19]:
custom_prefixes = nlp.Defaults.prefixes + (r"\$[a-zA-Z]+",)

TypeError: can only concatenate list (not "tuple") to list

In [ ]:
prefix_re = spacy.util.compile_prefix_regex(custom_prefixes)



In [ ]:
import re

prefix_re = re.compile(r"\$[a-zA-Z]+")
tokenizer = Tokenizer(
    nlp.vocab, prefix_search=prefix_re.search, suffix_search=suffix_re.search
)

tokens = tokenizer("This is a $STOCK.")
for token in tokens:
    print(token)

You can add also special-case tokenization rules. This mechanism is also used to add custom tokenizer exceptions to the language data. See the usage guide on the [languages data](https://spacy.io/usage/linguistic-features#language-data) and [tokenizer special cases](https://spacy.io/usage/linguistic-features#special-cases) for more details and examples.

In [ ]:
from spacy.attrs import ORTH, NORM, LOWER

dont_case = [{ORTH: "do"}, {ORTH: "n't", NORM: "not"}]
gimme_case = [{ORTH: "gi", NORM:"give"}, {ORTH: "me", NORM: "me"}]
tokenizer.add_special_case("don't", dont_case)
tokenizer.add_special_case("gimme", gimme_case)
tokens = tokenizer("Yo! gimme five!")
for token in tokens:
    print(token.norm_)
tokens = tokenizer("You don't do that")
for token in tokens:
    print(token.norm_)


When you load a model with pretrained NER (Named Entity Recognition), like `en_core_web_sm`, it is possible to make the tokenizer to merge the token for the entities it finds. Let's check what is inside the pipeline performed by `nlp`:


In [ ]:
nlp.pipeline

There's a tagger, a dependency parser and the entity recognizer. Let's check the entities of the following sentence:

In [ ]:
doc = nlp("Apple is a $1000b company.")


In [ ]:
for token in doc:
    print(token)

In [ ]:
for ent in doc.ents:
    print(ent, ent.label_)

In [ ]:
doc = nlp(
    "This is Strive School. It's worthy to merge 'Strive School' as a single token instead of two"
)

for token in doc:
    print(token)

In [ ]:
for ent in doc.ents:
    print(ent, ent.label_)

Let's add "merge_entities" to the pipeline (you can do it only if there is the entity recognizer):

In [ ]:
nlp.add_pipe(nlp.create_pipe("merge_entities"))

In [ ]:
nlp.pipeline

In [ ]:
doc = nlp(
    "This is Strive School. It's worthy to merge 'Strive School' as a single token instead of two"
)

for token in doc:
    print(token)

In [ ]:
TEXTS = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]


In [ ]:
for sentence in nlp.pipe(TEXTS):
    for token in sentence:
        print(token)
    print("------------------")

It's also possible to merge the noun chunks into one:

In [ ]:
nlp.add_pipe(nlp.create_pipe("merge_noun_chunks"))

In [ ]:
nlp.pipeline

In [ ]:
doc = nlp("Hello, I'm Antonio Marsella, nice to meet you.")
for token in doc:
    print(token)

## Removing stop words

In general, it's convenient to remove all the stop words, *i.e. very common words in a language*, because they don't help most of NLP problem such as semantic analysis.

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print("Number of stop words: %d" % len(spacy_stopwords))
print("First ten stop words: %s" % list(spacy_stopwords)[:10])

To remove them:

In [ ]:
text = """He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
indeed the vaguest idea where the wood and river in question were."""

doc = nlp(text)

tokens = [token.text for token in doc if not token.is_stop]
for token in tokens:
    print(token)

For adding customized stop words:

In [ ]:
customize_stop_words = ["computing", "filtered"]
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

## Stemming and Lemmatization

In most natural languages, a root word can have many variants. For example, the word ‘play’ can be used as ‘playing’, ‘played’, ‘plays’, etc. You can think of similar examples (and there are plenty).

**Stemming**

Let’s first understand stemming:

Stemming is a text normalization technique that cuts off the end or beginning of a word by taking into account a list of common prefixes or suffixes that could be found in that word
It is a rudimentary rule-based process of stripping the suffixes (“ing”, “ly”, “es”, “s” etc) from a word
 

**Lemmatization**

Lemmatization, on the other hand, is an organized & step-by-step procedure of obtaining the root form of the word. It makes use of vocabulary (dictionary importance of words) and morphological analysis (word structure and grammar relations).

Stemming algorithm works by cutting the suffix or prefix from the word. Lemmatization is a more powerful operation as it takes into consideration the morphological analysis of the word.

Lemmatization returns the lemma, which is the root word of all its inflection forms.

We can say that stemming is a quick and dirty method of chopping off words to its root form while on the other hand, lemmatization is an intelligent operation that uses dictionaries which are created by in-depth linguistic knowledge. Hence, Lemmatization helps in forming better features.

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(nlp.create_pipe("merge_entities"))
# not using merge_chunk_nouns
doc = nlp(
    u"""He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
indeed the vaguest idea where the wood and river in question were."""
)

lemma_word1 = []
for token in doc:
    if token.is_stop:
        continue
    lemma_word1.append(token.lemma_)
lemma_word1

## Removing the punctuation



In [ ]:
text = """He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
indeed the vaguest idea where the wood and river in question were."""


import string

text_no_punct = "".join([char for char in text if char not in string.punctuation])

text_no_punct

In [ ]:
doc = nlp(text_no_punct)
for token in doc:
    print(token)

For text extracted from dialogues or chats, it is convenient to preprocess the text so that multiple occurrences of the same characters get condensed into one or two, and then use a spell checker to find the correct form of the word.

A way to do that is to replace all the occurrences of repeated characters with a single one and then use a spell checker: "hhheeelllllooo hoooowww areee youuu?" becomes "helo how are you?" and then the spell checker would make it "hello how are you?"




In [ ]:
st = "hhheeeLLLLooo hoooowww areee youuu?????"
text = re.sub(r"(.)\1+", r"\1", st)
text

In [ ]:
from spellchecker import SpellChecker

text = nlp(text)
spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown([token.text for token in text])


for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

It didn't find any mispelled (even if there was "helo"). Try another spell checker:

https://github.com/fsondej/autocorrect

In [ ]:
from autocorrect import Speller

spell = Speller()

spell(text.text)

As you can see, it's not always working properly! However, overall it should improve your text.

If you want to create a separate lemmatizer instead of having it in the pipeline:

In [ ]:
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB

lemmatizer = nlp.vocab.morphology.lemmatizer
print(lemmatizer("studying", VERB))
print(lemmatizer("studying", NOUN))
print(lemmatizer("studying", ADJ))

# or as alternative

print(lemmatizer.verb("studying"))
print(lemmatizer.noun("studying"))
print(lemmatizer.adj("studying"))

spaCy has no built-in stemming! However, Lemmatization is enough for most of the tasks. As alternative, you can use [NLTK library](https://www.nltk.org).

## Part of Speech (POS) Tagging

Parts of speech tagging simply refers to assigning parts of speech to individual words in a sentence, which means that, unlike phrase matching, which is performed at the sentence or multi-word level, parts of speech tagging is performed at the token level.

In [ ]:
from collections import Counter, defaultdict

sentence = nlp("Antonio is learning Python in Strive School.")

for token in sentence:
    print(token.pos_)

The `.pos_` attribute gives the *coarse-grained* POS tag. To inspect the *fine-grained* POS tags we could use the `.tag_`attribute:

In [ ]:
sentence = nlp("Antonio is learning Python in Strive School.")

for token in sentence:
    print(token.tag_)

While the output of the `.pos_` attribute is easy to decrypt (`PROPN`: proper noun,
`AUX`: Auxiliary verb,
`VERB`: verb,
`ADP`: Adposition,
`PUNCT`: Punctuation), the `.tag_`'s output is more cryptic. For this, you can use the `spacy.explain()` function to get the intuition behind that:


In [ ]:
for token in sentence:
    print(spacy.explain(token.tag_))

Go and dig up your primary school grammar book!

Let's put everything together:

In [ ]:
for token in sentence:
    print(f'{token.text:{12}} {token.pos_:{10}} {token.tag_:{8}} {spacy.explain(token.tag_)}')


(the numbers between curly brackets define spaces for a better formatting).

You can count the number of occurrences of each POS tag by calling the `count_by` method. 

The syntax is as follows (you need to pass `spacy.attrs.POS` as argument of the method):

In [ ]:
sentence = nlp("Antonio is learning Python Programming Language")

num_pos = sentence.count_by(spacy.attrs.POS)
num_pos

The keys of the vocabulary are the ID of the POS tags, the values are their frequencies of occurrence. To retrieve the POS tags given the ID, you can do as follows:

In [ ]:
sentence.vocab[96].text

where 96 is the ID of the tag. Printing all together:

In [ ]:
for ID, frequency in num_pos.items():
    print(f"{ID} stands for {sentence.vocab[ID].text:{8}}: {frequency}")

## Named Entity Recognition

A named entity is a “real-world object” that’s assigned a name – for example, a person, a country, a product or a book title. spaCy can recognize various types of named entities in a document, by asking the model for a prediction. Because models are statistical and strongly depend on the examples they were trained on, this doesn’t always work perfectly and might need some tuning later, depending on your use case.

Named entities are available as the ents property of a Doc.


Example:



In [ ]:
doc = nlp("Antonio works at Strive School.")

In [ ]:
from spacy import displacy

displacy.render(doc, style="ent")

In [ ]:
doc = nlp("Rome is a big city.")

In [ ]:
displacy.render(doc, style="ent")

ORG stands for organization, GPE stands for Geopolitical Entity. Some other tags are:

In spaCy you can list the entities by doing:
    

In [ ]:
doc = nlp('Manchester United is looking to sign Harry Kane for $90 million')

In [ ]:
doc.ents

We can access the entities text, label by doing:

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Even if the entities are self-explanatory for this example, you can use `spacy.explain()` for a detailed description.

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_, spacy.explain(ent.label_))

### Adding new entities

If in a text an entity has not being identified correctly, you can manually add it as follows:

In [ ]:
sentence = nlp(u'Strive is setting up a new course.')
for entity in sentence.ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

In [ ]:
from spacy.tokens import Span

ORG = sentence.vocab.strings[u'ORG']

new_entity = Span(sentence, 0, 1, label=ORG)
sentence.ents = list(sentence.ents) + [new_entity]

In [ ]:
sentence.ents

First, we need to import the Span class from the `spacy.tokens` module. Next, we need to get the hash value of the ORG entity type from our document. After that, we need to assign the hash value of ORG to the span. Since "Strive" is the first word in the document, the span is 0-1. Finally, we need to add the new entity span to the list of entities. Now if you execute the following script, you will see "Strive" in the list of entities.

In [ ]:
for entity in sentence.ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

In [ ]:
from spacy import displacy

displacy.render(sentence, style="ent")

We can also filter which entity to display:

In [ ]:
sentence = nlp(u'Manchester United is looking to sign Harry Kane for $90 million. David demand 100 Million Dollars')
displacy.render(sentence, style='ent', jupyter=True)

In [ ]:
filter = {'ents': ['ORG']}
displacy.render(sentence, style='ent', jupyter=True, options=filter)

## Matcher

The Matcher lets you find words and phrases using rules describing their token attributes. Rules can refer to token annotations (like the text or part-of-speech tags), as well as lexical attributes like Token.is_punct. Applying the matcher to a Doc gives you access to the matched tokens in context. For in-depth examples and workflows for combining rules and statistical models, see the usage guide on rule-based matching.

https://spacy.io/api/matcher

Test the explorer: https://explosion.ai/demos/matcher

In [ ]:
from spacy.matcher import Matcher


Let’s say we want to enable spaCy to find a combination of three tokens:

- A token whose lowercase form matches “hello”, e.g. “Hello” or “HELLO”.
- A token whose is_punct flag is set to True, i.e. any punctuation.
- A token whose lowercase form matches “world”, e.g. “World” or “WORLD”.

The pattern would be:


In [ ]:
pattern = [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}]


We can define a custom matcher that satisfy that pattern by doing:

In [ ]:
matcher = Matcher(nlp.vocab)
matcher.add("HelloWorld", [pattern])

In [ ]:
doc = nlp("Hello, world! Hello world!")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(string_id, start, end, span.text)

In [ ]:
golang_pattern = [{"LOWER":{
    "IN":["go", "golang"]}, 
                   "POS":{"NOT_IN":["VERB"]}}]

In [20]:
doc = nlp("I go the learn the Go programming language")
matcher.add("GOLANG", [golang_pattern])
matches = matcher(doc)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(string_id, start, end, span.text)

NameError: name 'matcher' is not defined

In [ ]:
for token in doc:
    print(token.pos_)

In [ ]:
text = nlp("This is a tweet about Dogecoin $DOGE #DOGE")

for i, token in enumerate(text):
    print(i, token)

In [ ]:
pattern = [ {"ORTH":"$", "OP":"?"},
           {"ORTH":"#", "OP":"?"},
        {"LOWER": {"IN":["doge", "dogecoin"]}}
]

In [ ]:
matcher = Matcher(nlp.vocab)
matcher.add("DOGE", [pattern])

In [ ]:
matches = matcher(text)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = text[start:end]  # The matched span
    print(string_id, start, end, span.text)

You can also match "whatever token" by adding an empty \{\} in the pattern:

In [ ]:
text = nlp("I want to match whatever follows here")
pattern = [ {"LOWER":"follows"},

]

In [ ]:
matcher = Matcher(nlp.vocab)
matcher.add("WHATEVER", [pattern])

In [ ]:
matches = matcher(text)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = text[start:end]  # The matched span
    print(string_id, start, end, span.text)